In [ ]:
#Load libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
from random import randrange, uniform

In [ ]:
train = pd.read_csv(r"C:\Users\anupr\Desktop\churn\Train_data.csv")
test = pd.read_csv(r"C:\Users\anupr\Desktop\churn\Test_data.csv")

In [ ]:
train.shape
test.shape

In [ ]:
train.columns
test.columns

In [ ]:
train.dtypes
test.dtypes

In [ ]:
plt.boxplot(train['number vmail messages'])

In [ ]:
plt.boxplot(train['total day calls'])

In [ ]:
plt.boxplot(train['total night minutes'])

In [ ]:
#save numeric names
cnames =  ["account length","area code","number vmail messages","total day minutes", "total day calls", "total day charge", "total eve calls", "total eve charge", "total night minutes", "total night calls", "total night charge", "total intl minutes

In [ ]:
# #Detect and delete outliers from data
for i in cnames:
     print(i)
     q75, q25 = np.percentile(train.loc[:,i], [75 ,25])
     iqr = q75 - q25

     min = q25 - (iqr*1.5)
     max = q75 + (iqr*1.5)
     #print(min)
     #print(max)
    
     train = train.drop(train[train.loc[:,i] < min].index)
     train = train.drop(train[train.loc[:,i] > max].index)

In [ ]:
# #Detect and delete outliers from data
for i in cnames:
     print(i)
     q75, q25 = np.percentile(test.loc[:,i], [75 ,25])
     iqr = q75 - q25

     min = q25 - (iqr*1.5)
     max = q75 + (iqr*1.5)
     #print(min)
     #print(max)
    
     test = test.drop(test[test.loc[:,i] < min].index)
     test = test.drop(test[test.loc[:,i] > max].index)

In [ ]:
train.isnull().sum()

# Feature Selection

In [ ]:
##Correlation analysis
#Correlation plot
df_corr = train.loc[:,cnames]

In [ ]:
#Set the width and hieght of the plot
f, ax = plt.subplots(figsize=(7, 5))

#Generate correlation matrix
corr = df_corr.corr()

#Plot using seaborn library
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

In [ ]:
train = train.drop(['total day charge', 'total intl charge', 'total night charge'], axis=1)

In [ ]:
test = test.drop(['total day charge', 'total intl charge', 'total night charge'], axis=1)

In [ ]:
#Chisquare test of independence
#Save categorical variables
cat_names = ["state", "international plan", "voice mail plan"]

In [ ]:
#loop for chi square values
for i in cat_names:
    print(i)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(train['Churn'], train[i]))
    print(p)

In [ ]:
df = train.copy()
train = df.copy()

In [ ]:
#save continous names
cnames =  ["total day minutes", "total day calls", "total eve minutes", "total eve charge", "total night minutes", "total intl minutes"]

In [ ]:
def Normalise(df):
    for i in cnames:
        #print(i)
        df[i] = (df[i] - df[i].min())/(df[i].max()-df[i].min())
    return df


#Normalization 
train = Normalise(train)
test = Normalise(test)

In [ ]:
X = train.drop('Churn',1)
y = train.Churn

In [ ]:
X = test.drop('Churn',1)
y = test.Churn

In [ ]:
X=pd.get_dummies(X)
train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size =0.6)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
model = LogisticRegression()
model.fit(x_train, y_train)

In [ ]:
pred_cv = model.predict(x_cv)

In [ ]:
accuracy_score(y_cv,pred_cv)

In [ ]:
pred_test = model.predict(test)

In [ ]:
sample = pd.read_csv(r"C:\Users\anupr\Desktop\churn\Sample_output.csv")

In [ ]:
sample['Churn']=pred_test
sample['phone_number']=test_original['phone_number']

In [ ]:
sample['Churn'].replace(0, 'N',inplace=True)
sample['Churn'].replace(1, 'Y',inplace=True)

In [ ]:
pd.df(sample, columns=['phone_number','Churn']).to_csv('logistic.csv')

# Decision Tree

In [ ]:
from sklearn import tree

In [ ]:
i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
for train_index,test_index in kf.split(X,y):
     print('\n{} of kfold {}'.format(i,kf.n_splits))
     xtr,xvl = X.loc[train_index],X.loc[test_index]
     ytr,yvl = y[train_index],y[test_index]
    
     model = tree.DecisionTreeClassifier(random_state=1)
     model.fit(xtr, ytr)
     pred_test = model.predict(xvl)
     score = accuracy_score(yvl,pred_test)
     print('accuracy_score',score)
     i+=1
pred_test = model.predict(test)

In [ ]:
sample['Churn']=pred_test
sample['phone_number']=test_original['phone_number']

In [ ]:
sample['Churn'].replace(0, 'N',inplace=True)
sample['Churn'].replace(1, 'Y',inplace=True)

In [ ]:
pd.df(sample, columns=['phone_number','Churn']).to_csv('decesion_tree.csv')

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
for train_index,test_index in kf.split(X,y):
     print('\n{} of kfold {}'.format(i,kf.n_splits))
     xtr,xvl = X.loc[train_index],X.loc[test_index]
     ytr,yvl = y[train_index],y[test_index]
    
     model = RandomForestClassifier(random_state=1, max_depth=10)
     model.fit(xtr, ytr)
     pred_test = model.predict(xvl)
     score = accuracy_score(yvl,pred_test)
     print('accuracy_score',score)
     i+=1
pred_test = model.predict(test)


In [ ]:
sample['Churn']=pred_test
sample['phone_number']=test_original['phone_number']

In [ ]:
sample['Churn'].replace(0, 'N',inplace=True)
sample['Churn'].replace(1, 'Y',inplace=True)

In [ ]:
pd.df(sample, columns=['phone_number','Churn']).to_csv('random.csv')